In [ ]:
from PIL import Image
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import pickle
import random
from google.colab import drive
drive.mount('./gdrive', force_remount=True)

In [ ]:
# pickle 불러오기
with open('./gdrive/My Drive/Images/des_pickles/des_ecb_2r.pickle', 'rb') as f:
  ecb_pkl = pickle.load(f)
  f.close()
with open('./gdrive/My Drive/Images/des_pickles/des_cbc_2r.pickle', 'rb') as f:
  cbc_pkl = pickle.load(f)
  f.close()
with open('./gdrive/My Drive/Images/des_pickles/des_ofb_2r.pickle', 'rb') as f:
  ofb_pkl = pickle.load(f)

In [ ]:
# 정답 레이블
ecb_label = [0 for _ in range(1000)]
cbc_label = [1 for _ in range(1000)]
ofb_label = [2 for _ in range(1000)]
# label1 : ECB, CBC 1라운드 구분 모델 정답 레이블
# label2 : CBC, OFB 1라운드 구분 모델 정답 레이블
# label3 : ECB, OFB 1라운드 구분 모델 정답 레이블
# label4 : ECB, CBC, OFB 1라운드 구분 모델 정답 레이블
#label1 = np.array(ecb_label + cbc_label)
#label2 = np.array(cbc_label + ofb_label)
#label3 = np.array(ecb_label + ofb_label)
label4 = np.array(ecb_label + cbc_label + ofb_label)

In [ ]:
# data1 : ECB, CBC 1라운드 데이터 합치고 섞기
# data2 : CBC, OFB 1라운드 데이터 합치고 섞기
# data3 : ECB, OFB 1라운드 데이터 합치고 섞기
# data4 : ECB, CBC, OFB 1라운드 데이터 합치고 섞기

#data1 = np.concatenate((ecb_pkl, cbc_pkl), axis=0)
#data2 = np.concatenate((cbc_pkl, ofb_pkl), axis=0)
#data3 = np.concatenate((ecb_pkl, ofb_pkl), axis=0)
data4 = np.concatenate((ecb_pkl, cbc_pkl, ofb_pkl), axis=0)

#tmp1 = [[x, y] for x, y in zip(data1, label1)]
#tmp2 = [[x, y] for x, y in zip(data2, label2)]
#tmp3 = [[x, y] for x, y in zip(data3, label3)]
tmp4 = [[x, y] for x, y in zip(data4, label4)]

#random.shuffle(tmp1)
#random.shuffle(tmp2)
#random.shuffle(tmp3)
random.shuffle(tmp4)

#X1 = np.array([n[0] for n in tmp1])
#Y1 = np.array([n[1] for n in tmp1])
#X2 = np.array([n[0] for n in tmp2])
#Y2 = np.array([n[1] for n in tmp2])
#X3 = np.array([n[0] for n in tmp3])
#Y3 = np.array([n[1] for n in tmp3])
X4 = np.array([n[0] for n in tmp4])
Y4 = np.array([n[1] for n in tmp4])

In [ ]:
#x_valid1, x_train1 = X1[:200] / 255, X1[200:] / 255
#x_valid2, x_train2 = X2[:200] / 255, X2[200:] / 255
#x_valid3, x_train3 = X3[:200] / 255, X3[200:] / 255
x_valid4, x_train4 = X4[:300] / 255, X4[300:] / 255
#y_valid1, y_train1 = Y1[:200], Y1[200:]
#y_valid2, y_train2 = Y2[:200], Y2[200:]
#y_valid3, y_train3 = Y3[:200], Y3[200:]
y_valid4, y_train4 = Y4[:300], Y4[300:]

In [ ]:
model = keras.models.Sequential([
              keras.layers.Conv2D(32, kernel_size=(4,1), activation="relu", strides=(1,4), padding="same", input_shape=(128,128,3)),
              keras.layers.MaxPool2D(pool_size=(2,1), padding="valid"),
              keras.layers.Dropout(0.25),
              keras.layers.Conv2D(64, kernel_size=(4,1), activation="relu", strides=(1,4), padding="same"),
              keras.layers.MaxPool2D(pool_size=(2,1), padding="valid"),
              keras.layers.Dropout(0.25),
              keras.layers.Conv2D(128, kernel_size=(4,1), activation="relu", strides=(1,4), padding="same"),
              keras.layers.MaxPool2D(pool_size=(2,1), padding="valid"),
              keras.layers.Dropout(0.25),
              keras.layers.Flatten(),
              keras.layers.Dense(512, activation="relu"),
              keras.layers.Dropout(0.5),
              keras.layers.Dense(3, activation="softmax")
])

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics="accuracy")

In [ ]:
# history1 : data1, label1
# history2 : data2, label2
# history3 : data3, label3
# history4 : data4, label4

#history1 = model.fit(x_train1, y_train1, epochs=30, validation_data=(x_valid1, y_valid1))
#history2 = model.fit(x_train2, y_train2, epochs=30, validation_data=(x_valid2, y_valid2))
#history3 = model.fit(x_train3, y_train3, epochs=30, validation_data=(x_valid3, y_valid3))
history4 = model.fit(x_train4, y_train4, epochs=30, validation_data=(x_valid4, y_valid4))

In [ ]:
def accuracy_plot(history):
  acc = history.history['accuracy']
  val_acc = history.history['val_accuracy']
  loss = history.history['loss']
  val_loss = history.history['val_loss']
  plt.plot(acc, label="accuracy")
  plt.plot(val_acc, label="val_accuracy")
  plt.plot(loss, label="loss")
  plt.plot(val_loss, label="val_loss")
  plt.xlabel('epoch')
  plt.ylabel('ratio')
  plt.ylim(0, 1.0)
  plt.legend()
  plt.show()
  return

In [ ]:
#accuracy_plot(history1)
#accuracy_plot(history2)
#accuracy_plot(history3)
accuracy_plot(history4)